## Code generation

In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install sympy
# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install matplotlib

In [2]:
# Remove matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
# initialize sympy printing (for latex output)
from sympy import init_printing, Symbol
init_printing()

# import functions and classes for compartment models
import sys
sys.path.insert(0, "./compartor") #use local copy
from compartor import *

Here, we illustrate how to export the moment equations in LaTeX format or generate code for simulations.

We consider again the exemplary model of the paper and derive moments equations with the automated function.

In [4]:
x = Content('x')
y = Content('y')
z = Content('z')

from sympy import symbols, log, exp, sin, cos, tan
kM,h = symbols('kM h') # The Michaelis-Menten constant and hill exponent
kl = symbols('kl') # The leaking activation coeff

# define intake distribution
pi_I = OutcomeDistribution.Poisson(Symbol("\pi_{Poiss}(y; \lambda)"), y[0], Symbol("\lambda"))
# define fragmentation distribution
pi_F = OutcomeDistribution.Uniform(Symbol("\pi_F(y|x)"), y[0], 0, x[0])

def activation(x):
    return x[0]**h / (kM**h + x[0]**h)

Birth         = TransitionClass( [x]       -to> [x+1],       'k_b', g=kl+activation(x), name='b')
Death         = TransitionClass( [x]       -to> [x-1],       'k_d', g=x[0],    name='d')

Intake        = TransitionClass( {}        -to> [y],         'k_I',         pi=pi_I, name='I')
# Exit          = TransitionClass( [x]       -to> {},          'k_E',                  name='E')
Exit          = TransitionClass( [x] + [y] -to> [x], 'k_E', name='E')

# Division = TransitionClass( [x]       -to> [y] + [x-y], 'k_D', g=x[0], pi=pi_F, name='D')
Division = TransitionClass( [x]       -to> [y] + [x-y], 'k_D', g=kl+activation(x), pi=pi_F, name='D')
# Division = TransitionClass( [x]       -to> [(0,)] + [(0,)], 'k_D', g=x[0], pi=pi_F, name='D')

transitions = [Birth, Death, Intake, Exit, Division]
display_transition_classes(transitions)

           [x] ---> [(1,) + x],  h_b = k_b*(kl + x[0]**h/(kM**h + x[0]**h))*n(
                         [x] ---> [(-1,) + x],  h_d = k_d*n(x)*x[0]           
                   EmptySet ---> [y],  h_I = \pi_{Poiss}(y; \lambda)*k_I      
[x] + [y] ---> [x],  h_E = k_E*(n(y) - KroneckerDelta(x, y))*n(x)/(KroneckerDe
    [x] ---> [x - y] + [y],  h_D = \pi_F(y|x)*k_D*(kl + x[0]**h/(kM**h + x[0]*

x)            
              
              
lta(x, y) + 1)
*h))*n(x)     

In [5]:
moments = [
    Moment(0),
    Moment(0)**2,
    Moment(1),
    Moment(1)**2
]
display(moments)
equations = automated_moment_equations(1, transitions, moments, clna=True)
# equations = automated_moment_equations(1, transitions, moments, clna=False)
display_moment_equations(equations)

⎡                 2                      2   ⎤
⎣Moment(0), Moment (0), Moment(1), Moment (1)⎦

dfM: DeltaM(0,)
monomials: [(1, 1, DeltaM(0))]
contentFun: 0
contentFun_clna: 0
dfMdt: 0
contentFun: 0
contentFun_clna: 0
dfMdt: 0
contentFun: 1
contentFun_clna: 1
dfMdt: k_I
contentFun: -1
contentFun_clna: -1
CHECK BOOLS: alpha=[0], beta=[0], boolean_variables=set()
	l_n_Xc: -k_E
	replaced1: [-k_E*(2*(Moment(0) - E[Moment(0,)])*E[Moment(0,)] + E[Moment(0,)]**2)/2]
	replaced2: [-k_E*Moment(0)/2]
dfMdt: k_E*(-(2*Moment(0) - E[Moment(0,)])*E[Moment(0,)] + Moment(0))/2
contentFun: kl + x[0]**h/(kM**h + x[0]**h)
contentFun_clna: (h*kM**h*(E[Moment(1,)]/E[Moment(0,)])**h*E[Moment(0,)]*x[0] - h*kM**h*(E[Moment(1,)]/E[Moment(0,)])**h*E[Moment(1,)] + kM**(2*h)*kl*E[Moment(1,)] + 2*kM**h*kl*(E[Moment(1,)]/E[Moment(0,)])**h*E[Moment(1,)] + kM**h*(E[Moment(1,)]/E[Moment(0,)])**h*E[Moment(1,)] + kl*(E[Moment(1,)]/E[Moment(0,)])**(2*h)*E[Moment(1,)] + (E[Moment(1,)]/E[Moment(0,)])**(2*h)*E[Moment(1,)])/((kM**(2*h) + 2*kM**h*(E[Moment(1,)]/E[Moment(0,)])**h + (E[Moment(1,)]/E[Moment(0,)])**(2*h))*E[

dfMdt: 2*k_b*(-h*kM**h*(E[Moment(1,)]/E[Moment(0,)])**h*Moment(0)*E[Moment(1,)] + h*kM**h*(E[Moment(1,)]/E[Moment(0,)])**h*Moment(1)*E[Moment(0,)] + kM**(2*h)*kl*Moment(0)*E[Moment(1,)] + 2*kM**h*kl*(E[Moment(1,)]/E[Moment(0,)])**h*Moment(0)*E[Moment(1,)] + kM**h*(E[Moment(1,)]/E[Moment(0,)])**h*Moment(0)*E[Moment(1,)] + kl*(E[Moment(1,)]/E[Moment(0,)])**(2*h)*Moment(0)*E[Moment(1,)] + (E[Moment(1,)]/E[Moment(0,)])**(2*h)*Moment(0)*E[Moment(1,)])*Moment(1)/((kM**(2*h) + 2*kM**h*(E[Moment(1,)]/E[Moment(0,)])**h + (E[Moment(1,)]/E[Moment(0,)])**(2*h))*E[Moment(1,)])
contentFun: x[0]
contentFun_clna: x[0]
dfMdt: k_d*Moment(1)
contentFun: -x[0]
contentFun_clna: -x[0]
dfMdt: -2*k_d*Moment(1)**2
contentFun: \lambda**2 + \lambda
contentFun_clna: \lambda*(\lambda + 1)
dfMdt: \lambda*k_I*(\lambda + 1)
contentFun: \lambda
contentFun_clna: \lambda
dfMdt: 2*\lambda*k_I*Moment(1)
contentFun: y[0]**2
contentFun_clna: (2*E[Moment(0,)]*y[0] - E[Moment(1,)])*E[Moment(1,)]/E[Moment(0,)]**2
CHECK BOOLS: 

dfM: DeltaM(1)**2 + 2*DeltaM(1)*Moment(1)
monomials: [(1, 1, DeltaM(1)**2), (2, Moment(1), DeltaM(1))]
contentFun: kl + x[0]**h/(kM**h + x[0]**h)
contentFun_clna: (h*kM**h*(E[Moment(1,)]/E[Moment(0,)])**h*E[Moment(0,)]*x[0] - h*kM**h*(E[Moment(1,)]/E[Moment(0,)])**h*E[Moment(1,)] + kM**(2*h)*kl*E[Moment(1,)] + 2*kM**h*kl*(E[Moment(1,)]/E[Moment(0,)])**h*E[Moment(1,)] + kM**h*(E[Moment(1,)]/E[Moment(0,)])**h*E[Moment(1,)] + kl*(E[Moment(1,)]/E[Moment(0,)])**(2*h)*E[Moment(1,)] + (E[Moment(1,)]/E[Moment(0,)])**(2*h)*E[Moment(1,)])/((kM**(2*h) + 2*kM**h*(E[Moment(1,)]/E[Moment(0,)])**h + (E[Moment(1,)]/E[Moment(0,)])**(2*h))*E[Moment(1,)])
dfMdt: k_b*(-h*kM**h*(E[Moment(1,)]/E[Moment(0,)])**h*Moment(0)*E[Moment(1,)] + h*kM**h*(E[Moment(1,)]/E[Moment(0,)])**h*Moment(1)*E[Moment(0,)] + kM**(2*h)*kl*Moment(0)*E[Moment(1,)] + 2*kM**h*kl*(E[Moment(1,)]/E[Moment(0,)])**h*Moment(0)*E[Moment(1,)] + kM**h*(E[Moment(1,)]/E[Moment(0,)])**h*Moment(0)*E[Moment(1,)] + kl*(E[Moment(1,)]/E[Moment(0,)])**

Computed moment equations for desired moments $\left< N\right> $, $\left< N^{2}\right> $, $\left< M^{1}\right> $, and $\left< {\left(M^{1}\right)}^{\mathtt{\text{2}}}\right> $.

...

                                                                              
                                                                              
                                                   h                2        h
                       h                         kM ⋅k_E⋅Expectation (N)   kM 
                     kM ⋅k_D⋅kl⋅Expectation(N) - ─────────────────────── + ───
d                                                           2                 
──(Expectation(N)) = ─────────────────────────────────────────────────────────
dt                                                                            
                                                                              
                                                                              
                                                                              

                                                                              
                                                   

                                                                              
                                                                              
                                                          h                   
                            h     ⎛Expectation(Moment(1))⎞                    
                      2⋅h⋅kM ⋅k_D⋅⎜──────────────────────⎟ ⋅Expectation(N)⋅Exp
d ⎛           ⎛ 2⎞⎞               ⎝    Expectation(N)    ⎠                    
──⎝Expectation⎝N ⎠⎠ = ────────────────────────────────────────────────────────
dt                                                                            
                                                                              
                                                                              
                                                                              

                                                                              
                                                   

                                                                              
                                                                              
                                                                              
                                       h                   ⎛Expectation(Moment
                             \lambda⋅kM ⋅k_I + \lambda⋅k_I⋅⎜──────────────────
d                                                          ⎝    Expectation(N)
──(Expectation(Moment(1))) = ─────────────────────────────────────────────────
dt                                                                            
                                                                              
                                                                              
                                                                              

                                                                              
                                                   

                                                                              
                                                                              
                                                                              
                                     2   2⋅h                                  
                              \lambda ⋅kM   ⋅k_I⋅Expectation(N)⋅Expectation(Mo
d ⎛           ⎛      2   ⎞⎞                                                   
──⎝Expectation⎝Moment (1)⎠⎠ = ────────────────────────────────────────────────
dt                                                                            
                                                                              
                                                                              
                                                                              

                                                                              
                                                   

                                                                              
                                                                              
                                                                              
                                         2⋅h                                  
                               \lambda⋅kM   ⋅k_I⋅Expectation(N)⋅Expectation(Mo
d                                                                             
──(Expectation(N⋅Moment(1))) = ───────────────────────────────────────────────
dt                                                                            
                                                                              
                                                                              
                                                                              

                                                                              
                                                   

The LaTeX source of the ODE system can be found and copy-pasted by modifying the Math Render option of the jupyter notebook. This is done by right-clicking on the system and choosing Math Settings > Math Render > Plain Source .

From the closed equations, we can also generate code to simulate the system. Currently, Python or Julia code can be generated.

For a direct code output, the user can rely on the functions:

In [6]:
julia_code = generate_julia_code(equations, function_name="SimpleSwitch")
print(julia_code)

# evaluate ODEs
function SimpleSwitch_ODEs(dM, M, parameters, t)
  c1 = parameters[:\lambda] # \lambda
  c5 = parameters[:h] # h
  c3 = parameters[:kM] # kM
  c6 = parameters[:kD] # kD
  c4 = parameters[:kE] # kE
  c7 = parameters[:kI] # kI
  c8 = parameters[:kb] # kb
  c2 = parameters[:kd] # kd
  c0 = parameters[:kl] # kl
  # Number of Compartments (N)
  dM[1] = (c7*c3^c5+c7*(M[3]/M[1])^c5+c6*(M[3]/M[1])^c5*M[1]+1/2*c4*c3^c5*M[1]+1/2*c4*(M[3]/M[1])^c5*M[1]-1/2*c4*c3^c5*M[1]^2-1/2*c4*(M[3]/M[1])^c5*M[1]^2+c0*c6*c3^c5*M[1]+c0*c6*(M[3]/M[1])^c5*M[1])/(c3^c5+(M[3]/M[1])^c5)
  # N^2
  dM[2] = (c7*c3^(2*c5)*M[3]+c7*(M[3]/M[1])^(2*c5)*M[3]+c6*(M[3]/M[1])^(2*c5)*M[1]*M[3]+c4*c3^(2*c5)*M[1]^3*M[3]+c4*c3^(2*c5)*M[2]*M[3]+c4*(M[3]/M[1])^(2*c5)*M[1]^3*M[3]+c4*(M[3]/M[1])^(2*c5)*M[2]*M[3]+1/2*c4*c3^(2*c5)*M[1]^2*M[3]+1/2*c4*(M[3]/M[1])^(2*c5)*M[1]^2*M[3]+2*c6*(M[3]/M[1])^(2*c5)*M[2]*M[3]+2*c7*c3^c5*(M[3]/M[1])^c5*M[3]+2*c7*c3^(2*c5)*M[1]*M[3]+2*c7*(M[3]/M[1])^(2*c5)*M[1]*M[3]-1/2*c4*c3^(2*c5)*M[1]